In [1]:
import warnings
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies import b_testing_strats
from src.utils import load_data

warnings.filterwarnings("ignore")
pd.set_option("display.float_format", "{:,.2f}".format)

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
ticker = "BTC-USD"
period = "max"
stock = yf.Ticker(ticker.upper())
data = stock.history(period=period)

In [3]:
data.dropna()
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2014-09-17 00:00:00+00:00,465.86,468.17,452.42,457.33,21056800,0.00,0.00
2014-09-18 00:00:00+00:00,456.86,456.86,413.10,424.44,34483200,0.00,0.00
2014-09-19 00:00:00+00:00,424.10,427.83,384.53,394.80,37919700,0.00,0.00
2014-09-20 00:00:00+00:00,394.67,423.30,389.88,408.90,36863600,0.00,0.00
2014-09-21 00:00:00+00:00,408.08,412.43,393.18,398.82,26580100,0.00,0.00
...,...,...,...,...,...,...,...
2025-03-09 00:00:00+00:00,"86,154.30","86,471.13","80,052.48","80,601.04",30899345977,0.00,0.00
2025-03-10 00:00:00+00:00,"80,597.15","83,955.93","77,420.59","78,532.00",54061099422,0.00,0.00
2025-03-11 00:00:00+00:00,"78,523.88","83,577.76","76,624.25","82,862.21",54702837196,0.00,0.00


In [4]:
strategy = b_testing_strats.SmaCross
strategy.n1 = 50
strategy.n2 = 125
bt_sma = Backtest(data, strategy, cash=10_000, commission=0.004, trade_on_close=True)
stats = bt_sma.run()
stats

Start                     2014-09-17 00:00...
End                       2025-03-13 00:00...
Duration                   3830 days 00:00:00
Exposure Time [%]                       58.47
Equity Final [$]                 1,273,222.36
Equity Peak [$]                  1,622,483.18
Return [%]                          12,632.22
Buy & Hold Return [%]               17,546.73
Return (Ann.) [%]                       58.69
Volatility (Ann.) [%]                   94.50
Sharpe Ratio                             0.62
Sortino Ratio                            1.59
Calmar Ratio                             0.87
Max. Drawdown [%]                      -67.42
Avg. Drawdown [%]                      -12.09
Max. Drawdown Duration     1316 days 00:00:00
Avg. Drawdown Duration       63 days 00:00:00
# Trades                                   11
Win Rate [%]                            63.64
Best Trade [%]                       1,319.99
Worst Trade [%]                        -33.38
Avg. Trade [%]                    

In [5]:
bt_sma.plot()

Row(id='1450', ...)

In [6]:
%%time
stats = bt_sma.optimize(
    n1=range(20, 70, 10),
    n2=range(100, 250, 20),
    maximize="Equity Final [$]",
    constraint=lambda param: param.n1 < param.n2,
)
stats

  0%|          | 0/14 [00:00<?, ?it/s]

CPU times: user 3.34 s, sys: 57.8 ms, total: 3.39 s
Wall time: 3.42 s


Start                     2014-09-17 00:00...
End                       2025-03-13 00:00...
Duration                   3830 days 00:00:00
Exposure Time [%]                       58.21
Equity Final [$]                 2,861,388.41
Equity Peak [$]                  3,342,592.21
Return [%]                          28,513.88
Buy & Hold Return [%]               17,546.73
Return (Ann.) [%]                       71.42
Volatility (Ann.) [%]                  102.04
Sharpe Ratio                             0.70
Sortino Ratio                            1.95
Calmar Ratio                             1.08
Max. Drawdown [%]                      -65.93
Avg. Drawdown [%]                      -10.72
Max. Drawdown Duration     1055 days 00:00:00
Avg. Drawdown Duration       52 days 00:00:00
# Trades                                   13
Win Rate [%]                            61.54
Best Trade [%]                       1,448.87
Worst Trade [%]                        -29.46
Avg. Trade [%]                    

In [7]:
bt_sma.plot()

Row(id='2182', ...)

In [8]:
strategy = b_testing_strats.RSICross
strategy.low_threshold = 36
strategy.high_threshold = 80
bt_rsi = Backtest(data, strategy, cash=10_000, commission=0.004, trade_on_close=True)
stats = bt_rsi.run()
stats

Start                     2014-09-17 00:00...
End                       2025-03-13 00:00...
Duration                   3830 days 00:00:00
Exposure Time [%]                       41.43
Equity Final [$]                    19,496.91
Equity Peak [$]                     54,699.90
Return [%]                              94.97
Buy & Hold Return [%]               17,546.73
Return (Ann.) [%]                        6.57
Volatility (Ann.) [%]                   44.13
Sharpe Ratio                             0.15
Sortino Ratio                            0.24
Calmar Ratio                             0.08
Max. Drawdown [%]                      -79.37
Avg. Drawdown [%]                      -10.86
Max. Drawdown Duration     2609 days 00:00:00
Avg. Drawdown Duration      157 days 00:00:00
# Trades                                  227
Win Rate [%]                            37.00
Best Trade [%]                         123.23
Worst Trade [%]                        -37.42
Avg. Trade [%]                    

In [9]:
bt_rsi.plot()

Row(id='2940', ...)

In [10]:
%%time
stats = bt_rsi.optimize(
    low_threshold=range(20, 41, 2),
    high_threshold=range(60, 81, 2),
    maximize="Equity Final [$]",
)
stats

  0%|          | 0/13 [00:00<?, ?it/s]

CPU times: user 12.8 s, sys: 171 ms, total: 12.9 s
Wall time: 13 s


Start                     2014-09-17 00:00...
End                       2025-03-13 00:00...
Duration                   3830 days 00:00:00
Exposure Time [%]                       38.40
Equity Final [$]                    70,172.83
Equity Peak [$]                     89,980.50
Return [%]                             601.73
Buy & Hold Return [%]               17,546.73
Return (Ann.) [%]                       20.40
Volatility (Ann.) [%]                   47.46
Sharpe Ratio                             0.43
Sortino Ratio                            0.83
Calmar Ratio                             0.34
Max. Drawdown [%]                      -59.15
Avg. Drawdown [%]                       -9.55
Max. Drawdown Duration     1405 days 00:00:00
Avg. Drawdown Duration       92 days 00:00:00
# Trades                                  130
Win Rate [%]                            40.77
Best Trade [%]                          76.51
Worst Trade [%]                        -23.51
Avg. Trade [%]                    

In [11]:
stats._strategy

<Strategy RSICross(low_threshold=30,high_threshold=70)>

In [12]:
strategy = b_testing_strats.PriceStrength
strategy.lookback_period_days = 50
strategy.hold_period_in_days = 2
bt_price_strength = Backtest(data, strategy, cash=10_000, commission=0.004, trade_on_close=True)
stats = bt_price_strength.run()
stats

RuntimeError: Indicator "recent_al…(C,50)" errored with exception: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices